In [1]:
import os
import re
files =  os.listdir()
pdf_file = next(x for x in files if x[-3:] =='pdf')

In [2]:
import pdfplumber

with pdfplumber.open(pdf_file) as pdf:
    first_page = pdf.pages[0]
    print(first_page.chars[0])

{'matrix': (0.750673792721924, 0.0, 0.0, 0.750673792721924, 115.38311937736982, 735.4743976832335), 'fontname': 'AAAAAA+ZoramldsLat-Regular', 'adv': 10.966640624999998, 'upright': True, 'x0': 115.38311937736982, 'y0': 731.6631154094432, 'x1': 123.6154890887569, 'y1': 744.6722922373142, 'width': 8.23236971138708, 'height': 13.009176827870988, 'size': 13.009176827870988, 'object_type': 'char', 'page_number': 1, 'text': 'A', 'stroking_color': (0.1294, 0.1333, 0.1451), 'non_stroking_color': (0.1294, 0.1333, 0.1451), 'top': 47.327707762685804, 'bottom': 60.33688459055679, 'doctop': 47.327707762685804}


In [3]:
households = []
texts_flattened = []
household_id = 0
for page_i, page in enumerate(pdf.pages):
    page_households = []
    household = []
    page_cont=True
    for i, word in enumerate(page.extract_words()):
        if word['x0']==31.75336715694298:
            page_households.append(household)
            household=[]
            word['type']='last_name'
            household.append(word)
        elif not word['text']=='©' and page_cont == True:
            household.append(word)
        else:
            page_cont = False
            pass
    page_households.append(household)

    del page_households[0]
    
    for i, household in enumerate(page_households):
        for ii, text in enumerate(household):
            page_households[i][ii]['id']= f'{page_i}-{i}'
            page_households[i][ii]['page_id']=page_i
            page_households[i][ii]['household_id']=household_id
            household[ii]['localx0']=text['x0']-household[0]['x0']
            household[ii]['localx1']=text['x1']-household[0]['x1']
            household[ii]['localtop']=text['top']-household[0]['top']
            household[ii]['localbottom']=text['bottom']-household[0]['bottom']
            char=list(filter(lambda char: char['x0'] == text['x0'] and char['top']==text['top'], page.chars))[0]
            household[ii]['fontname']=char['fontname']
            household[ii]['size']=char['size']
            texts_flattened.append(household[ii])
        household_id+=1
    households.extend(page_households)

In [4]:
class household:
    def __init__(self,items_list):
        self.items_list = items_list
        self.name_list=[]
        self.address_list=[]
        self.feature_loop()
        self.find_lat_lon()
        self.split_address()

    def within_address_box(self, text_dict):
        localx0 = 185.66274539392109,
        localtop = -1.6303696435678603,
        localx1 = 250,
        localbottom = 32.10517445703567
        if text_dict['localx0']>= 185.66274539392109 \
            and text_dict['localtop'] >=-1.6304\
            and text_dict['localx1']<=250\
            and text_dict['localbottom'] <= 32.10517445703567:
            self.address_list.append(text_dict['text'])
            return True
        else:
            return False
    
    def within_name_box(self, text_dict):
        localx0 = 185.66274539392109,
        localtop = -1.6303696435678603,
        localx1 = 250,
        localbottom = 32.10517445703567
        if text_dict['localx0']>= 0 \
            and text_dict['localtop'] >=0\
            and text_dict['localx1']<=180\
            and text_dict['localbottom'] <= 0:
            self.name_list.append(text_dict['text'])
            return True
        else:
            return False
    
    def feature_loop(self):
        for item in self.items_list:
            self.within_address_box(item)
            self.within_name_box(item)
        self.address =' '.join(self.address_list)
        self.name =' '.join(self.name_list)
    
    def find_lat_lon(self):
        self.lat_pattern = '[0-9]{2}[.][0-9]{1,6}'
        self.lon_pattern = '[-][0-9][0-9][.][0-9]{1,6}'
        self.lat = re.findall(self.lat_pattern,self.address)
        self.lon = re.findall(self.lon_pattern,self.address)
        if self.lat:
            self.lat = float(self.lat[0])
        else:
            self.lat = None
        if self.lon:
            self.lon = float(self.lon[0])
        else:
            self.lon=None
            
    def split_address(self):
        self.lat_pattern = '[0-9]{2}[.][0-9]{1,6}'
        self.address = re.split(self.lat_pattern,self.address)[0]
    
    def dump(self):
        self.out = {
            'name':self.name,
            'address':self.address,
            'lat':self.lat,
            'lon':self.lon
        }
        return self.out

In [47]:
out = []
for hh in households:
    house=household(hh)
    out.append(house.dump())

In [48]:
import pandas as pd
df = pd.DataFrame(out)
df['cagegory']=0

In [135]:
df.to_csv('needs_categorization.csv',index=False)
df.to_clipboard(index=False)

In [50]:
df = pd.read_csv('with_categorization.csv',usecols=['name', 'address', 'lat', 'lon', 'category'])

In [158]:
df_input=pd.read_clipboard()

In [164]:
df_input

,name,address,lat,lon,category
0,"Acajabon, Leah P","25 Denrose Dr Apt 10 Amherst, New York",43.013286,-78.819474,3
1,"Aisen, Aaron & Lauren","8325 Clarnew Dr East Amherst, New York",43.011423,-78.683771,1
2,"Antonetty, Juan Mateo","Member moved out from 1375 #5P Gettzville, New...",NaN,NaN,3
3,"Bagan, Kristie Kreuzer","Member moved out from Apt J East Amherst, New ...",43.032923,-78.705590,3
4,"Barnes, Nathan","Member moved out from Williamsville, New York",42.960075,-78.733605,3
...,...,...,...,...,...
148,"Young, Ann","137 Shire Dr N E Amherst, New York",43.001419,-78.707314,2
149,"Young, James","137 Shire Drive North East Amherst, New York",43.001398,-78.707340,2
150,"Zorn, Michael & Sheryl","172 Belvoir Rd Williamsville, New York",42.982080,-78.730725,3
151,"Zou, Biao & Guanwen",286 Wellington Ave. Bu,42.967038,-78.833995,2


In [160]:
def make_assignements(df_param, included_categories): 
    df_=df_param.copy()
    X=df_[(df_['lat']>0) & (df_['category'].isin(included_categories))][['lat','lon']].values
    mapper=df_[(df_['lat']>0) & (df_['category'].isin(included_categories))][['lat','lon']].index
    print(len(X))
    from sklearn.cluster import KMeans
    import numpy as np

    kmeans = KMeans(n_clusters=13, random_state=0).fit(X)
    kmeans.labels_

    Y =kmeans.cluster_centers_
    
    from h3 import point_dist
    distances = []
    for y in Y:
        out=[]
        for x in X:
            out.append(point_dist((y),(x),unit='km'))
        out = pd.Series(out,mapper)
        distances.append(out)
        
    out = []
    out_pairs=[]
    while len(distances[-1])>0:
        first=[]
        for i,district in enumerate(distances):
            if not district.empty:
                current_min = district.idxmin()
                first.append(current_min)
                df_.loc[current_min,'district']=i
                for ii, value in enumerate(distances):
                    distances[ii].drop(current_min,inplace=True)
            else:
                pass
        #print(first)
        out.append(first)
    assignments = pd.DataFrame(out)
    
    return df_, assignments

In [165]:
primary_df, primary_assignments = make_assignements(df_input,[1])
primary_df.to_clipboard(index=False)


40


In [163]:
prim_sec_df, prim_sec_assignments= make_assignements(df_input,[1,2])
prim_sec_df.to_clipboard(index=False)

61


In [73]:
#df[df['name']=='Lambson, Paul & Suvi Catriona']
df[df['name']=='Stark, Robert & Jennilyn']


,name,address,lat,lon,category,district
126,"Stark, Robert & Jennilyn","12 Firethorn Ct E Amherst, New York",43.016951,-78.712887,1,5.0


In [74]:
df[df['district']==3]

,name,address,lat,lon,category,district
8,"Biesinger, Zy & Jackie","817 W Center St Medina, New York 14103-1037",43.220452,-78.401975,1,3.0
9,"Blatto, Anthony & Joy",4944 Meadowbrook Road Bu,42.981762,-78.671543,2,3.0
28,"De Almeida, Marcelo","9061 Calpin Pkwy Clarence Center,",42.999999,-78.653130,3,3.0
31,DeGra,"10865 Howe Rd Clarence, New York",42.999519,-78.582932,1,3.0
47,"Haentges, Sean & Melissa","8947 Candlewood lane clarence center, New York",43.013367,-78.658244,3,3.0
55,"Hulse, Ryan & Amy","220 Stuewe Rd. Getzville, New York 14068",43.015715,-78.753686,1,3.0
77,"Litman, Mr. & Diana","10665 Clarence Center Rd Clarence, New York",43.005587,-78.590769,3,3.0
91,"Ossenberg, Erich","32B Linda Lane Cheektowaga, New York 14225",42.931235,-78.757189,2,3.0
118,"Showalter, Tyler",9625,42.978471,-78.634244,3,3.0
123,"Smith, Raymond Alexander","10500 Bergtold Rd Clarence, New York",42.967067,-78.598393,3,3.0


In [108]:
df.to_clipboard(index=False)

In [109]:
df

,name,address,lat,lon,category,district
0,"Acajabon, Leah P","25 Denrose Dr Apt 10 Amherst, New York",43.013286,-78.819474,3,8.0
1,"Aisen, Aaron & Lauren","8325 Clarnew Dr East Amherst, New York",43.011423,-78.683771,1,10.0
2,"Antonetty, Juan Mateo","Member moved out from 1375 #5P Gettzville, New...",NaN,NaN,3,NaN
3,"Bagan, Kristie Kreuzer","Member moved out from Apt J East Amherst, New ...",43.032923,-78.705590,3,5.0
4,"Barnes, Nathan","Member moved out from Williamsville, New York",42.960075,-78.733605,3,0.0
...,...,...,...,...,...,...
148,"Young, Ann","137 Shire Dr N E Amherst, New York",43.001419,-78.707314,2,1.0
149,"Young, James","137 Shire Drive North East Amherst, New York",43.001398,-78.707340,2,12.0
150,"Zorn, Michael & Sheryl","172 Belvoir Rd Williamsville, New York",42.982080,-78.730725,3,7.0
151,"Zou, Biao & Guanwen",286 Wellington Ave. Bu,42.967038,-78.833995,2,7.0


In [123]:
included_categories = [1,2]
X=df[(df['lat']>0) & (df['category'].isin(included_categories))][['lat','lon']].values
mapper=df[(df['lat']>0) & (df['category'].isin(included_categories))][['lat','lon']].index
print(len(X))
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=13, random_state=0).fit(X)
kmeans.labels_

Y =kmeans.cluster_centers_

from h3 import point_dist
distances = []
for y in Y:
    out=[]
    for x in X:
        out.append(point_dist((y),(x),unit='km'))
    out = pd.Series(out,mapper)
    distances.append(out)

out = []
out_pairs=[]
while len(distances[-1])>0:
    first=[]
    for i,district in enumerate(distances):
        if not district.empty:
            current_min = district.idxmin()
            first.append(current_min)
            df.loc[current_min,'district']=i
            for ii, value in enumerate(distances):
                distances[ii].drop(current_min,inplace=True)
        else:
            pass
    #print(first)
    out.append(first)
assignments = pd.DataFrame(out)

58
